### Twitter Text Classifier
#### Step 1.
Import Libraries 
1. Pandas: Used for structured data operations and manipulations. 
2. Numpy:Stands for numerical python. It works on n-dimensional array.
3. nltk:
4. sklearn:


In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

np.random.seed(500)


In [3]:
Corpus = pd.read_csv(r"corpus.csv",encoding='latin-1')

In [6]:
# Step - a : Remove blank rows if any.
Corpus['text'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Corpus['text'] = [entry.lower() for entry in Corpus['text']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]

In [10]:
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Corpus['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [11]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)

In [68]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [56]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [54]:
a = Train_X_Tfidf.toarray()

In [65]:
print(Train_X_Tfidf)

  (0, 4506)	0.3763188267807246
  (0, 4505)	0.15031494427382475
  (0, 3976)	0.35868777245753825
  (0, 3890)	0.2515140235472667
  (0, 3860)	0.2690675584422277
  (0, 3745)	0.34695623926050195
  (0, 3655)	0.2896999547088821
  (0, 3564)	0.29449641491430995
  (0, 2931)	0.229683025366997
  (0, 1935)	0.13406125327954532
  (0, 1529)	0.17761496997588844
  (0, 521)	0.321056290554803
  (0, 491)	0.12303572865008613
  (0, 240)	0.2448559358109696
  (1, 4687)	0.21384275526442909
  (1, 4060)	0.3566872275481094
  (1, 3434)	0.21279175847748263
  (1, 3313)	0.8157357261127677
  (1, 2592)	0.2173336717856602
  (1, 1260)	0.2074693534878867
  (1, 604)	0.1614401835472762
  (2, 4734)	0.18368024895699514
  (2, 4625)	0.15009324791624393
  (2, 4466)	0.10284196288022608
  (2, 4191)	0.1168175572462934
  :	:
  (6998, 2523)	0.11512525241790023
  (6998, 2125)	0.13650351320774795
  (6998, 1971)	0.07126979525506368
  (6998, 1778)	0.22013576217543665
  (6998, 1746)	0.19935227823153098
  (6998, 1710)	0.13509114757747032
  (

In [67]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  51.26666666666667
